### Shuffling


In [ ]:
import os
import random
import pandas as pd
from pathlib import Path

# Define the path to your images directory
images_dir = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/Label_Studio/all_images")

# Get all image files
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith('.png')]
print(f"Found {len(image_files)} images")

# Shuffle the list
random.seed(42)  # For reproducibility
random.shuffle(image_files)

# Create a DataFrame with the shuffled list
df = pd.DataFrame({'filename': image_files})

# Save to CSV in your code directory
output_dir = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/Label_Studio/code")
shuffled_list_file = output_dir / "shuffled_images.csv"
df.to_csv(shuffled_list_file, index=False)

# Display the first 10 shuffled images
print(f"\nShuffled list saved to {shuffled_list_file}")
print("\nFirst 10 images in shuffled list:")
df.head(10)

In [1]:
import os
from pathlib import Path

# Use the correct path to your shuffled images
shuffled_images_dir = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/shuffled_images")

# Check if directory exists
if not os.path.exists(shuffled_images_dir):
    print(f"Directory does not exist: {shuffled_images_dir}")
else:
    print(f"Found image directory: {shuffled_images_dir}")
    
    # Get all image files (limit to first 20 to avoid memory issues)
    image_files = [f for f in os.listdir(shuffled_images_dir) if f.lower().endswith('.png')]
    print(f"Found {len(image_files)} PNG images")
    
    # Print sample filenames
    print("\nSample filenames:")
    for filename in image_files[:5]:
        print(f"  {filename}")
    
    # Analyze one filename in detail
    if image_files:
        print("\nDetailed analysis of first filename:")
        filename = image_files[0]
        print(f"  Filename: {filename}")
        parts = filename.split('_')
        print(f"  Parts after splitting by underscore: {parts}")
        
        # Extract potential style (last part before extension)
        if len(parts) > 1:
            last_part = parts[-1]
            print(f"  Last part: {last_part}")
            style = last_part.split('.')[0]
            print(f"  Style (last part without extension): {style}")
            
        # Find all unique styles (analyzing more files)
        styles = {}
        for i, filename in enumerate(image_files):
            if i >= 100:  # Limit to first 100 files to avoid memory issues
                break
                
            parts = filename.split('_')
            if len(parts) > 1:
                style = parts[-1].split('.')[0]
                styles[style] = styles.get(style, 0) + 1
        
        print(f"\nFound {len(styles)} potential styles in first 100 images:")
        for style, count in styles.items():
            print(f"  {style}: {count} files")

Found image directory: C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\shuffled_images
Found 3935 PNG images

Sample filenames:
  0001_simpa_simpa_173_artistic.png
  0002_asset_asset_125_technical.png
  0003_simpa_simpa_239_cartoon.png
  0004_simpa_simpa_398_cartoon.png
  0005_asset_asset_137_realistic.png

Detailed analysis of first filename:
  Filename: 0001_simpa_simpa_173_artistic.png
  Parts after splitting by underscore: ['0001', 'simpa', 'simpa', '173', 'artistic.png']
  Last part: artistic.png
  Style (last part without extension): artistic

Found 10 potential styles in first 100 images:
  artistic: 16 files
  technical: 14 files
  cartoon: 8 files
  realistic: 11 files
  digital art: 8 files
  storybook: 7 files
  3d rendered: 12 files
  geometric: 7 files
  retro: 7 files
  minimalistic: 10 files


### Creating the 2000 set

In [4]:
import os
import shutil
import pandas as pd
from pathlib import Path
from collections import defaultdict

# Define paths - update these to match your environment
SOURCE_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/shuffled_images")
TARGET_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/2000_set")

# Create target directory if it doesn't exist
os.makedirs(TARGET_DIR, exist_ok=True)

# Get all image files from source directory
image_files = [f for f in os.listdir(SOURCE_DIR) if f.lower().endswith('.png')]
print(f"Found {len(image_files)} source images")

# Group images by their dataset_ID combination
image_groups = defaultdict(list)

for img_file in image_files:
    # Parse filename to extract components
    # Format: ####_dataset_dataset_ID_style.png
    parts = img_file.split('_')
    
    if len(parts) >= 4:  # Make sure there are enough parts
        # The dataset_ID key will be dataset_dataset_ID
        dataset_id_key = f"{parts[1]}_{parts[2]}_{parts[3]}"
        image_groups[dataset_id_key].append(img_file)

# Check how many unique base images we have
print(f"Found {len(image_groups)} unique base images")

# Verify that each base image has 10 style variations
complete_groups = {k: v for k, v in image_groups.items() if len(v) == 10}
print(f"Found {len(complete_groups)} base images with all 10 style variations")

# Select 200 complete groups (base images with 10 styles each)
selected_groups = list(complete_groups.keys())[:200]
print(f"Selected {len(selected_groups)} base images for our 2000_set")

# Create a mapping file to keep track of selected images
selected_image_mapping = []

# Copy selected images to target directory with error handling
copied_count = 0
errors = []

for group_key in selected_groups:
    image_files = image_groups[group_key]
    for img_file in image_files:
        source_path = SOURCE_DIR / img_file
        target_path = TARGET_DIR / img_file
        
        try:
            # Copy the file
            shutil.copy2(str(source_path), str(target_path))  # Convert Path to string
            copied_count += 1
            
            # Add to mapping
            selected_image_mapping.append({
                'original_filename': img_file,
                'base_image_id': group_key,
                'style': img_file.split('_')[-1].split('.')[0]  # Extract style
            })
        except Exception as e:
            errors.append(f"Error copying {img_file}: {str(e)}")

print(f"Copied {copied_count} images to {TARGET_DIR}")
if errors:
    print(f"Encountered {len(errors)} errors:")
    for error in errors[:5]:  # Show just the first 5 errors
        print(f"  {error}")

# If we have successful copies, save the mapping
if copied_count > 0:
    # Save the mapping to a CSV file
    mapping_df = pd.DataFrame(selected_image_mapping)
    mapping_path = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/code/2000_set_mapping.csv")
    mapping_df.to_csv(mapping_path, index=False)
    print(f"Saved mapping to {mapping_path}")

    # Count styles in selected set
    style_counts = mapping_df['style'].value_counts()
    print("\nStyle distribution in selected set:")
    print(style_counts)

Found 3935 source images
Found 400 unique base images
Found 362 base images with all 10 style variations
Selected 200 base images for our 2000_set
Copied 2000 images to C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\2000_set
Saved mapping to C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\code\2000_set_mapping.csv

Style distribution in selected set:
style
artistic        200
digital art     200
realistic       200
geometric       200
storybook       200
retro           200
technical       200
cartoon         200
3d rendered     200
minimalistic    200
Name: count, dtype: int64


Let's add a verification code to check if all 10 styles for a specific base image (like simpa_173) are in the 2000_set folder:


In [6]:
import os
from pathlib import Path

# Define the path to the 2000_set directory
target_dir = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/2000_set")

# Check if the directory exists
if not os.path.exists(target_dir):
    print(f"Directory does not exist: {target_dir}")
else:
    print(f"Found target directory: {target_dir}")
    
    # Get all image files
    image_files = [f for f in os.listdir(target_dir) if f.lower().endswith('.png')]
    print(f"Found {len(image_files)} images in the 2000_set folder")
    
    # Check for a specific base image (e.g., simpa_173)
    base_image_to_check = "wikipedia_wikipedia_395"
    matching_files = [f for f in image_files if base_image_to_check in f]
    print(f"\nChecking styles for base image: {base_image_to_check}")
    if matching_files:
        print(f"Found {len(matching_files)} style variations:")
        for file in matching_files:
            # Extract the style from the filename
            style = file.split('_')[-1].split('.')[0]
            print(f"  - {style}")
    else:
        print(f"No images found for base image: {base_image_to_check}")
        
    # Let's check a few random base images
    print("\nChecking a few random base images:")
    
    # Extract all unique base image IDs
    base_image_ids = set()
    for img_file in image_files:
        parts = img_file.split('_')
        if len(parts) >= 4:
            base_id = f"{parts[1]}_{parts[2]}_{parts[3]}"
            base_image_ids.add(base_id)
    
    # Check the first 5 base images (or fewer if there are less than 5)
    for i, base_id in enumerate(list(base_image_ids)[:5]):
        matching_files = [f for f in image_files if base_id in f]
        print(f"\nBase image {i+1}: {base_id}")
        print(f"  Found {len(matching_files)} style variations")
        if len(matching_files) != 10:
            print("  Styles found:")
            for file in matching_files:
                style = file.split('_')[-1].split('.')[0]
                print(f"    - {style}")

Found target directory: C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\2000_set
Found 2000 images in the 2000_set folder

Checking styles for base image: wikipedia_wikipedia_395
Found 10 style variations:
  - digital art
  - technical
  - 3d rendered
  - minimalistic
  - cartoon
  - retro
  - realistic
  - geometric
  - artistic
  - storybook

Checking a few random base images:

Base image 1: wikipedia_wikipedia_197
  Found 10 style variations

Base image 2: onestopenglish_onestop_058
  Found 10 style variations

Base image 3: simpa_simpa_122
  Found 10 style variations

Base image 4: simpa_simpa_422
  Found 10 style variations

Base image 5: onestopenglish_onestop_235
  Found 10 style variations


### renaming 2000

In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path

# Define paths
SOURCE_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/2000_set")
TARGET_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/2000_set_renamed")
MAPPING_FILE = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/code/renamed_images_mapping.csv")

# Create target directory if it doesn't exist
os.makedirs(TARGET_DIR, exist_ok=True)

# Get all image files from source directory
image_files = [f for f in os.listdir(SOURCE_DIR) if f.lower().endswith('.png')]
print(f"Found {len(image_files)} images in 2000_set")

# Create a list to store the mapping information
mapping_data = []

# Copy and rename the files with error handling
copied_count = 0
errors = []

for img_file in image_files:
    try:
        # Extract the sequence number (####) from the original filename
        sequence_number = img_file.split('_')[0]
        
        # Create new filename (just the sequence number with extension)
        new_filename = f"{sequence_number}.png"
        
        # Define source and target paths
        source_path = SOURCE_DIR / img_file
        target_path = TARGET_DIR / new_filename
        
        # Copy the file with the new name
        shutil.copy2(str(source_path), str(target_path))
        copied_count += 1
        
        # Parse additional metadata from the filename
        parts = img_file.split('_')
        dataset = parts[1] if len(parts) > 1 else ""
        dataset_id = parts[3] if len(parts) > 3 else ""
        style = parts[4].split('.')[0] if len(parts) > 4 else ""
        
        # Add to mapping data
        mapping_data.append({
            'new_filename': new_filename,
            'original_filename': img_file,
            'sequence_number': sequence_number,
            'dataset': dataset,
            'dataset_id': dataset_id,
            'style': style,
            'base_image': f"{dataset}_{dataset}_{dataset_id}"
        })
    except Exception as e:
        errors.append(f"Error processing {img_file}: {str(e)}")

# Create DataFrame from mapping data
mapping_df = pd.DataFrame(mapping_data)

# Save mapping to CSV
mapping_df.to_csv(MAPPING_FILE, index=False)

# Verify the results
print(f"Renamed and copied {copied_count} images to {TARGET_DIR}")
print(f"Created mapping file at {MAPPING_FILE}")

if errors:
    print(f"Encountered {len(errors)} errors:")
    for error in errors[:5]:  # Show just the first 5 errors
        print(f"  {error}")

# Show a sample of the mapping for verification
print("\nSample of mapping file (first 5 rows):")
print(mapping_df.head())

# Check for any duplicate new filenames
duplicate_check = mapping_df['new_filename'].duplicated()
if duplicate_check.any():
    print(f"\nWARNING: Found {duplicate_check.sum()} duplicate new filenames")
    print(mapping_df[duplicate_check])
else:
    print("\nNo duplicate filenames found - everything looks good!")

# Count how many unique base images we have
unique_base_images = mapping_df['base_image'].nunique()
print(f"\nTotal unique base images: {unique_base_images}")

# Verify each base image has 10 styles
base_image_counts = mapping_df['base_image'].value_counts()
incomplete_base_images = base_image_counts[base_image_counts != 10]

if len(incomplete_base_images) > 0:
    print(f"\nFound {len(incomplete_base_images)} base images without exactly 10 styles:")
    print(incomplete_base_images)
else:
    print("\nAll base images have exactly 10 style variations - perfect!")

Found 2000 images in 2000_set
Renamed and copied 2000 images to C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\renamed_images
Created mapping file at C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\code\renamed_images_mapping.csv

Sample of mapping file (first 5 rows):
  new_filename                   original_filename sequence_number dataset  \
0     0001.png   0001_simpa_simpa_173_artistic.png            0001   simpa   
1     0002.png  0002_asset_asset_125_technical.png            0002   asset   
2     0003.png    0003_simpa_simpa_239_cartoon.png            0003   simpa   
3     0004.png    0004_simpa_simpa_398_cartoon.png            0004   simpa   
4     0005.png  0005_asset_asset_137_realistic.png            0005   asset   

  dataset_id      style       base_image  
0        173   artistic  simpa_simpa_173  
1        125  technical  asset_asset_125  
2        239    cartoon  simpa_simpa_239  
3        398    cartoon  simpa_simpa

Expert Distribution

* First, we'll select 200 images for the IAA set from the renamed_images folder
* Then, create the expert distribution structure with 4 folders named after each expert
* Copy the IAA set to each expert's folder
* Distribute 450 unique images to each expert's unique set folder

In [8]:
import os
import shutil
import pandas as pd
import random
import numpy as np
from pathlib import Path

# Define paths
RENAMED_IMAGES_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/renamed_images")
EXPERT_DISTRIBUTION_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/Expert_distribution")
IAA_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/IAA_set")
MAPPING_FILE = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/code/renamed_images_mapping.csv")
DISTRIBUTION_MAPPING_FILE = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/code/expert_distribution_mapping.csv")

# Expert names
EXPERTS = [
    "Martin Kappus",
    "Luisa Carrer",
    "Katrin Andermatt",
    "Alexa Lintner"
]

# Create necessary directories
os.makedirs(EXPERT_DISTRIBUTION_DIR, exist_ok=True)
os.makedirs(IAA_DIR, exist_ok=True)

for expert in EXPERTS:
    expert_dir = EXPERT_DISTRIBUTION_DIR / expert
    os.makedirs(expert_dir, exist_ok=True)
    os.makedirs(expert_dir / "IAA_set", exist_ok=True)
    os.makedirs(expert_dir / "unique_set", exist_ok=True)

# Load the mapping file
try:
    mapping_df = pd.read_csv(MAPPING_FILE)
    print(f"Loaded mapping file with {len(mapping_df)} entries")
except Exception as e:
    print(f"Error loading mapping file: {str(e)}")
    exit(1)

# Get all unique base images
unique_base_images = mapping_df['base_image'].unique()
print(f"Found {len(unique_base_images)} unique base images")

# Select 20 base images for IAA (will result in 200 images with 10 styles each)
random.seed(42)  # For reproducibility
iaa_base_images = random.sample(list(unique_base_images), 20)
print(f"Selected {len(iaa_base_images)} base images for IAA")

# Get all images for the IAA set (the full 10 styles for each selected base image)
iaa_mask = mapping_df['base_image'].isin(iaa_base_images)
iaa_images_df = mapping_df[iaa_mask].copy()
iaa_images_df['is_iaa'] = True

# Mark the remaining images as non-IAA
non_iaa_images_df = mapping_df[~iaa_mask].copy()
non_iaa_images_df['is_iaa'] = False

# Combine back into a single dataframe
updated_mapping_df = pd.concat([iaa_images_df, non_iaa_images_df])

# Distribute the non-IAA images evenly among the 4 experts
non_iaa_base_images = list(set(non_iaa_images_df['base_image'].unique()))
print(f"Distributing {len(non_iaa_base_images)} non-IAA base images among {len(EXPERTS)} experts")

# Split the base images into 4 equal groups
np.random.seed(42)  # For reproducibility
np.random.shuffle(non_iaa_base_images)
expert_base_images = np.array_split(non_iaa_base_images, len(EXPERTS))

# Create a dictionary to track which expert gets which base images
expert_assignments = {}
for i, expert in enumerate(EXPERTS):
    expert_assignments[expert] = list(expert_base_images[i])
    print(f"{expert}: {len(expert_assignments[expert])} base images assigned")

# Add expert assignment to the mapping dataframe
updated_mapping_df['assigned_expert'] = None

for expert, base_images in expert_assignments.items():
    mask = updated_mapping_df['base_image'].isin(base_images)
    updated_mapping_df.loc[mask, 'assigned_expert'] = expert

# Copy IAA images to the IAA_DIR and to each expert's IAA folder
for _, row in iaa_images_df.iterrows():
    source_path = RENAMED_IMAGES_DIR / row['new_filename']
    
    # Add _IAA suffix to the filename for clarity
    iaa_filename = row['new_filename'].replace('.png', '_IAA.png')
    
    # Copy to main IAA directory
    target_path = IAA_DIR / iaa_filename
    shutil.copy2(str(source_path), str(target_path))
    
    # Copy to each expert's IAA folder
    for expert in EXPERTS:
        expert_iaa_path = EXPERT_DISTRIBUTION_DIR / expert / "IAA_set" / iaa_filename
        shutil.copy2(str(source_path), str(expert_iaa_path))

print(f"Copied {len(iaa_images_df)} IAA images to the IAA directory and each expert's IAA folder")

# Copy unique images to each expert's unique folder
for expert in EXPERTS:
    expert_images_df = updated_mapping_df[(updated_mapping_df['assigned_expert'] == expert) & (~updated_mapping_df['is_iaa'])]
    
    for _, row in expert_images_df.iterrows():
        source_path = RENAMED_IMAGES_DIR / row['new_filename']
        target_path = EXPERT_DISTRIBUTION_DIR / expert / "unique_set" / row['new_filename']
        shutil.copy2(str(source_path), str(target_path))
    
    print(f"Copied {len(expert_images_df)} unique images to {expert}'s unique folder")

# Save the updated mapping with IAA and expert assignments
updated_mapping_df.to_csv(DISTRIBUTION_MAPPING_FILE, index=False)
print(f"Saved distribution mapping to {DISTRIBUTION_MAPPING_FILE}")

# Verification
print("\nVerification:")
print(f"Total images in mapping: {len(updated_mapping_df)}")
print(f"IAA images: {updated_mapping_df['is_iaa'].sum()}")
print(f"Non-IAA images: {(~updated_mapping_df['is_iaa']).sum()}")

for expert in EXPERTS:
    expert_count = (updated_mapping_df['assigned_expert'] == expert).sum()
    expert_iaa_dir = EXPERT_DISTRIBUTION_DIR / expert / "IAA_set"
    expert_unique_dir = EXPERT_DISTRIBUTION_DIR / expert / "unique_set"
    
    iaa_files = len([f for f in os.listdir(expert_iaa_dir) if f.lower().endswith('.png')])
    unique_files = len([f for f in os.listdir(expert_unique_dir) if f.lower().endswith('.png')])
    
    print(f"\n{expert}:")
    print(f"  Assigned in mapping: {expert_count}")
    print(f"  Files in IAA folder: {iaa_files}")
    print(f"  Files in unique folder: {unique_files}")
    print(f"  Total files: {iaa_files + unique_files}")

Loaded mapping file with 2000 entries
Found 200 unique base images
Selected 20 base images for IAA
Distributing 180 non-IAA base images among 4 experts
Martin Kappus: 45 base images assigned
Luisa Carrer: 45 base images assigned
Katrin Andermatt: 45 base images assigned
Alexa Lintner: 45 base images assigned
Copied 200 IAA images to the IAA directory and each expert's IAA folder
Copied 450 unique images to Martin Kappus's unique folder
Copied 450 unique images to Luisa Carrer's unique folder
Copied 450 unique images to Katrin Andermatt's unique folder
Copied 450 unique images to Alexa Lintner's unique folder
Saved distribution mapping to C:\Users\SouayedBelkiss\OneDrive - gae\Desktop\Thesis\phase3_label_studio\code\expert_distribution_mapping.csv

Verification:
Total images in mapping: 2000
IAA images: 200
Non-IAA images: 1800

Martin Kappus:
  Assigned in mapping: 450
  Files in IAA folder: 200
  Files in unique folder: 450
  Total files: 650

Luisa Carrer:
  Assigned in mapping: 450


a focused verification code to confirm the distribution is correct

In [9]:
import os
from pathlib import Path

# Define paths
EXPERT_DISTRIBUTION_DIR = Path("C:/Users/SouayedBelkiss/OneDrive - gae/Desktop/Thesis/phase3_label_studio/Expert_distribution")

# Expert names
EXPERTS = [
    "Martin Kappus",
    "Luisa Carrer",
    "Katrin Andermatt",
    "Alexa Lintner"
]

print("Verifying image distribution across experts...")

# Dictionary to track all unique images seen
all_unique_images = set()
has_overlap = False

# Verify each expert's folders
for expert in EXPERTS:
    expert_dir = EXPERT_DISTRIBUTION_DIR / expert
    iaa_dir = expert_dir / "IAA_set"
    unique_dir = expert_dir / "unique_set"
    
    # Count IAA images
    iaa_images = [f for f in os.listdir(iaa_dir) if f.lower().endswith('.png')]
    iaa_count = len(iaa_images)
    
    # Count unique images
    unique_images = [f for f in os.listdir(unique_dir) if f.lower().endswith('.png')]
    unique_count = len(unique_images)
    
    # Check for overlaps in unique images
    unique_image_set = set(unique_images)
    overlaps = unique_image_set.intersection(all_unique_images)
    
    if overlaps:
        has_overlap = True
        print(f"WARNING: {expert} shares {len(overlaps)} unique images with other experts")
    
    # Add current expert's unique images to the set of all seen images
    all_unique_images.update(unique_image_set)
    
    # Report counts
    print(f"{expert}:")
    print(f"  IAA images: {iaa_count}/200 {'✓' if iaa_count == 200 else '✗'}")
    print(f"  Unique images: {unique_count}/450 {'✓' if unique_count == 450 else '✗'}")

# Final verification
if not has_overlap:
    print("\nVerification PASSED: Each expert has a completely unique set of 450 images!")
else:
    print("\nVerification FAILED: Some experts share unique images!")

Verifying image distribution across experts...
Martin Kappus:
  IAA images: 200/200 ✓
  Unique images: 450/450 ✓
Luisa Carrer:
  IAA images: 200/200 ✓
  Unique images: 450/450 ✓
Katrin Andermatt:
  IAA images: 200/200 ✓
  Unique images: 450/450 ✓
Alexa Lintner:
  IAA images: 200/200 ✓
  Unique images: 450/450 ✓

Verification PASSED: Each expert has a completely unique set of 450 images!
